# Window-funktioner

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-01-11

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om window-funktioner i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/t-sql/queries/select-over-clause-transact-sql?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/t-sql/queries/select-over-clause-transact-sql?view=sql-server-ver16)


## Indholdsfortegnelse

- [Introduktion til window-funktioner](#Introduktion-til-window-funktioner)
- [Opbygningen af window-funktioner](#Opbygningen-af-window-funktioner)
- [Logisk processering af window-funktioner](#Logisk-processering-af-window-funktioner)
- [Familier af window-funktioner](#Familier-af-window-funktioner)
- [Hovedpointer](#Hovedpointer)

## Introduktion til window-funktioner

*Windowing* bruges som et effektivt og fleksibelt værktøj til analytiske formål.

**Window-funktioner er, i sin essens, nogle funktioner som for hver række laver en beregning over en mængde af rækker relateret til den aktuelle række.**

Mængden af rækker som der laves en beregning henover, kaldes et vindue (*window*)

Modsat vores grupperede forespørgsler, så kan vi med window-funktioner bevare detaljen. Derudover kan man også definere en orden i sin beregning.

Use cases:

- Beregning af `TOP N` per gruppe
- Beregning af løbende totaler
- Dublethåndtering
- Beregning af overlap i intervaller
- Identifikation af huller og øer


In [1]:
/* Nedenstående er et eksempel på hvordan windowing tillader aggregering, men samtidig kan
bevare detaljen, modsat en grupperet forespørgsel */

CREATE TABLE #TableA (
    Id int NOT NULL,
    Dat date NOT NULL,
    Cat nvarchar(100) NOT NULL,
    Val int NULL
);

INSERT INTO #TableA (Id, Dat, Cat, Val)
VALUES
(1, '20220101', 'A', 22), (2, '20000504', 'A', 5), (3, '20150205', 'A', 0),
(4, '20101203', 'B', 14), (5, '20050824', 'B', 79), (6, '20220930', 'B', 100),
(7, '20220315', 'B', 43), (8, '20210710', 'B', 43), (9, '20000504', 'C', 1),
(10, '20221231', 'C', 112);

SELECT
    *
FROM #TableA;

/* En grupperet forespørgsel (hvor detaljen går tabt): */

SELECT
    Cat,
    SUM(Val) AS TotalVal
FROM #TableA
GROUP BY Cat
ORDER BY Cat;

/* En forespørgsel der bruger windowing (hvor detaljen bevares): */

SELECT
    Id,
    Dat,
    Cat,
    Val,
    SUM(Val) OVER (PARTITION BY Cat) AS TotalVal,
    1.0 * Val / SUM(Val) OVER (PARTITION BY Cat) AS PctVal
FROM #TableA
ORDER BY Cat, Dat, Id;

/* Nedenstående er et eksempel på hvordan man med window-funktioner også kan definere orden,
fx en rank */

SELECT
    Id,
    Cat,
    Val,
    RANK() OVER (
        PARTITION BY Cat
        ORDER BY Val DESC
    ) AS RankVal
FROM #TableA
ORDER BY Cat, Val DESC;

DROP TABLE #TableA;


(10 rows affected)

(10 rows affected)

(3 rows affected)

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.045

Id,Dat,Cat,Val
1,2022-01-01,A,22
2,2000-05-04,A,5
3,2015-02-05,A,0
4,2010-12-03,B,14
5,2005-08-24,B,79
6,2022-09-30,B,100
7,2022-03-15,B,43
8,2021-07-10,B,43
9,2000-05-04,C,1
10,2022-12-31,C,112


Cat,TotalVal
A,27
B,279
C,113


Id,Dat,Cat,Val,TotalVal,PctVal
2,2000-05-04,A,5,27,0.185185185185
3,2015-02-05,A,0,27,0.000000000000
1,2022-01-01,A,22,27,0.814814814814
5,2005-08-24,B,79,279,0.283154121863
4,2010-12-03,B,14,279,0.050179211469
8,2021-07-10,B,43,279,0.154121863799
7,2022-03-15,B,43,279,0.154121863799
6,2022-09-30,B,100,279,0.358422939068
9,2000-05-04,C,1,113,0.008849557522
10,2022-12-31,C,112,113,0.991150442477


Id,Cat,Val,RankVal
1,A,22,1
2,A,5,2
3,A,0,3
6,B,100,1
5,B,79,2
7,B,43,3
8,B,43,3
4,B,14,5
10,C,112,1
9,C,1,2


### *Tid til opgaver...*

Lav opgave 1 og 2 i [opgavehæftet](Window-functions.sql).


## Opbygningen af window-funktioner

Window-funktioner er overordnet opbygget af to elementer:

1. Funktionen, eller beregningen, som ønskes foretaget, fx `COUNT()`, `RANK()` eller `FIRST_VALUE()`
2. Specifikationen af selve vinduet i `OVER`-delsætningen

En window-funktion har følgende form:

```sql
SELECT
    <funktion> OVER (
        PARTITION BY <opdelingskolonner>
        ORDER BY <sorteringskolonner>
        ROWS/RANGE BETWEEN <øvre grænse> AND <nedre grænse>
    ) AS Beregning
```

`OVER`-delsætningen giver følgende muligheder for at specificere vinduet:

- *Partitioning*: Bruges til at opdele forespørgslen i grupper som beregningen foretages for. Hvis ikke denne angives, så laves beregningen for hele forespørgslen
- *Ordering*: Bruges til at bestemme ordenen som rækker evalueres i inden for en window-frame
- *Framing*: Bruges til udvælge en delmængde af rækker inden for en window-partiton

Bemærk, at ikke alle muligheder kan tages i brug for alle funktioner.


In [2]:
/* Nedenstående er et eksempel på brugen af en window-funktion, specifikt hvorledes resultatet
ændres når specifikationen af vinduet ændres */

CREATE TABLE #TableA (
    Id int NOT NULL,
    Dat date NOT NULL,
    Cat nvarchar(100) NOT NULL,
    Val int NULL
);

INSERT INTO #TableA (Id, Dat, Cat, Val)
VALUES
(1, '20220101', 'A', 22), (2, '20000504', 'A', 5), (3, '20150205', 'A', 0),
(4, '20101203', 'B', 14), (5, '20050824', 'B', 79), (6, '20220930', 'B', 100),
(7, '20220315', 'B', 43), (8, '20210710', 'B', 43), (9, '20000504', 'C', 1),
(10, '20221231', 'C', 112);

SELECT
    *
FROM #TableA;

/* Prøv at udskifte den aktuelle ORDER BY-delsætning med den udkommenterede. Hvorledes ændres
resultatet? */

SELECT
    Id,
    Cat,
    Dat,
    Val,
    ROW_NUMBER() OVER (
        PARTITION BY Cat
        ORDER BY Dat, Id
        --ORDER BY Dat DESC, Id DESC
    ) AS RowNum
FROM #TableA
ORDER BY Cat, Dat, Id;

/* Nedenstående er et eksempel på en løbende total. Der er en implicit window-frame:
RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW */

/* Hvad sker der hvis du ændrer window-framen ved indkommentere kommentaren markeret med -- ? */

SELECT
    Id,
    Cat,
    Dat,
    Val,
    SUM(Val) OVER (
        PARTITION BY Cat
        ORDER BY Dat, Id
        /*RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW*/
        --ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) AS RunningTotalVal
FROM #TableA
ORDER BY Cat, Dat, Id;

DROP TABLE #TableA;

(10 rows affected)

(10 rows affected)

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.054

Id,Dat,Cat,Val
1,2022-01-01,A,22
2,2000-05-04,A,5
3,2015-02-05,A,0
4,2010-12-03,B,14
5,2005-08-24,B,79
6,2022-09-30,B,100
7,2022-03-15,B,43
8,2021-07-10,B,43
9,2000-05-04,C,1
10,2022-12-31,C,112


Id,Cat,Dat,Val,RowNum
2,A,2000-05-04,5,1
3,A,2015-02-05,0,2
1,A,2022-01-01,22,3
5,B,2005-08-24,79,1
4,B,2010-12-03,14,2
8,B,2021-07-10,43,3
7,B,2022-03-15,43,4
6,B,2022-09-30,100,5
9,C,2000-05-04,1,1
10,C,2022-12-31,112,2


Id,Cat,Dat,Val,RunningTotalVal
2,A,2000-05-04,5,5
3,A,2015-02-05,0,5
1,A,2022-01-01,22,27
5,B,2005-08-24,79,79
4,B,2010-12-03,14,93
8,B,2021-07-10,43,136
7,B,2022-03-15,43,179
6,B,2022-09-30,100,279
9,C,2000-05-04,1,1
10,C,2022-12-31,112,113


### *Tid til opgaver...*

Lav opgave 3 i [opgavehæftet](Window-functions.sql).


## Logisk processering af window-funktioner

I vores logiske processering af en forespørgsel evalueres window-funktioner på følgende trin (markeret med fed):

1. `FROM`
2. `WHERE`
3. `GROUP BY`
4. `HAVING`
5. `SELECT`
   1. **Evaluering af udtryk**
   2. Fjernelse af dubletter
6. **`ORDER BY`**
7. `OFFSET-FETCH`/`TOP`

Dvs. i udgangspunktet understøttes window-funktioner udelukkende i `SELECT`- og `ORDER BY`-delsætningerne. Denne begrænsning skal sikre entydighed omkring hvilken underliggende tabel der laves beregninger på.

Hvis vi ønsker at bruge window-funktioner i andre delsætninger, så bliver vi nødt til at lave beregninger trinvist.


In [1]:
/* Nedenstående er nogle eksempler hvor vigtigheden af window-funktioners placering i den
logiske processering understreges */

CREATE TABLE #TableA (
    Id int NOT NULL,
    Col date NOT NULL
);

INSERT INTO #TableA (Id, Col)
VALUES
(1, '20220502'), (2, '20220314'), (3, '20230101'), (4, '20220314'), (5, '20220630');

SELECT
    *
FROM #TableA;

/* Placer window-funktionen i forskellige delsætninger ved at indkommentere linjerne en ad gangen.
Virker alt i overensstemmelse med den logiske processering? */

SELECT
    Id,
    Col,
    ROW_NUMBER() OVER (ORDER BY Col, Id) AS RowNum
FROM #TableA
--WHERE ROW_NUMBER() OVER (ORDER BY Col, Id) = 1
--ORDER BY ROW_NUMBER() OVER (ORDER BY Col, Id)
;
GO

/* Hvorfor er det mon problematisk hvis window-funktionen kunne placeres i fx WHERE-delsætningen? */

/* Et tænkt eksempel:
Hvad vil forespørgslen nedenfor returnere? 3 eller 3 og 5?
Og hvad hvis du skiftede den aktuelle WHERE-delsætning ud med den udkommenterde?
Husk på princippet om all-at-once operationer!
Det ville være forvirrende, ikke? */

SELECT
    Id
FROM #TableA
WHERE Col > '20220501'
    AND ROW_NUMBER() OVER (ORDER BY Col, Id) > 2
--WHERE ROW_NUMBER() OVER (ORDER BY Col, Id) > 2
--    AND Col > '20220501'
;
GO

/* Kan du gætte resultatet af nedenstående query? Virker det i overensstemmelse med
den logiske processering? */

SELECT DISTINCT
    Col,
    ROW_NUMBER() OVER(ORDER BY Col) AS RowNum
FROM #TableA
ORDER BY Col;

/* Hvordan kan vi komme uden om begrænsingen forårsaget af den logiske query processering?
Fx kan en CTE benyttes. */

WITH CTE AS (
    SELECT
        Id,
        Col,
        ROW_NUMBER() OVER(ORDER BY Col, Id) AS RowNum
    FROM #TableA
)

SELECT
    *
FROM CTE
WHERE RowNum > 2;

DROP TABLE #TableA;

(5 rows affected)

(5 rows affected)

(5 rows affected)

(5 rows affected)

(3 rows affected)

...

Id,Col
1,2022-05-02
2,2022-03-14
3,2023-01-01
4,2022-03-14
5,2022-06-30


Id,Col,RowNum
2,2022-03-14,1
4,2022-03-14,2
1,2022-05-02,3
5,2022-06-30,4
3,2023-01-01,5


: Msg 4108, Level 15, State 1, Line 42
Windowed functions can only appear in the SELECT or ORDER BY clauses.

Col,RowNum
2022-03-14,1
2022-03-14,2
2022-05-02,3
2022-06-30,4
2023-01-01,5


Id,Col,RowNum
1,2022-05-02,3
5,2022-06-30,4
3,2023-01-01,5


## Familier af window-funktioner

Der findes forskellige familier af window-funktioner:

- [Ranking](https://learn.microsoft.com/en-us/sql/t-sql/functions/ranking-functions-transact-sql?view=sql-server-ver16)
- [Offset](https://learn.microsoft.com/en-us/sql/t-sql/functions/analytic-functions-transact-sql?view=sql-server-ver16)
- [Aggregering](https://learn.microsoft.com/en-us/sql/t-sql/functions/aggregate-functions-transact-sql?view=sql-server-ver16)
- (Statistik)


In [2]:
/* Nedenfor vises beregninger som benytter de mest anvendte funktioner fra hver af de tre
familier  */

CREATE TABLE #TableA (
    Id int NOT NULL,
    Dat date NOT NULL,
    Cat nvarchar(100) NOT NULL,
    Val int NULL
);

INSERT INTO #TableA (Id, Dat, Cat, Val)
VALUES
(1, '20220101', 'A', 22), (2, '20000504', 'A', 5), (3, '20150205', 'A', 0),
(4, '20101203', 'B', 14), (5, '20050824', 'B', 79), (6, '20220930', 'B', 100),
(7, '20220315', 'B', 43), (8, '20210710', 'B', 43), (9, '20000504', 'C', 1),
(10, '20221231', 'C', 112);

SELECT
    *
FROM #TableA;

/* Ranking: */

SELECT
    Id,
    Dat,
    Cat,
    Val,
    ROW_NUMBER() OVER(
        PARTITION BY Cat
        ORDER BY Val
    ) AS RowNum,
    RANK() OVER(
        PARTITION BY Cat
        ORDER BY Val
    ) AS Rank,
    DENSE_RANK() OVER(
        PARTITION BY Cat
        ORDER BY Val
    ) AS DenseRank,
    NTILE(2) OVER(
        PARTITION BY Cat
        ORDER BY Val
    ) AS NTile
FROM #TableA
ORDER BY Cat, Val;

/* Bemærk, at ranking-funktioner ikke understøtter en window-frame */

/* Offset: */

SELECT
    Id,
    Dat,
    Cat,
    Val,
    LAG(Val/*, 1, NULL*/) OVER(
        PARTITION BY Cat
        ORDER BY Val
    ) AS PrevVal,
    LEAD(Val/*, 1, NULL*/) OVER(
        PARTITION BY Cat
        ORDER BY Val
    ) AS NextVal,
    FIRST_VALUE(Val) OVER(
        PARTITION BY Cat
        ORDER BY Val
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS FirstVal,
    LAST_VALUE(Val) OVER(
        PARTITION BY Cat
        ORDER BY Val
        ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING
    ) AS LastVal
FROM #TableA
ORDER BY Cat, Val;

/* Bemærk, at LAG() og LEAD() ikke understøtter en window-frame. Derudover kan man for disse
funktioner yderligere specificere en offset værdi (default er 1) og en værdi i tilfælde af NULL */

/* Aggregering: */

SELECT
    Id,
    Dat,
    Cat,
    Val,
    COUNT(*) OVER(
        PARTITION BY Cat
        /*ORDER BY Val
        RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW*/
    ) AS Cnt,
    MIN(Val) OVER(
        PARTITION BY Cat
        /*ORDER BY Val
        RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW*/
    ) AS MinVal,
    MAX(Val) OVER(
        PARTITION BY Cat
        /*ORDER BY Val
        RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW*/
    ) AS MaxVal,
    SUM(Val) OVER(
        PARTITION BY Cat
        /*ORDER BY Val
        RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW*/
    ) AS SumVal
FROM #TableA
ORDER BY Cat, Val;

/* Bemærk, at aggregeringsfunktionerne kan blive til fx løbende totaler ved at introducere
en window-order og -frame */

DROP TABLE #TableA;


(10 rows affected)

(10 rows affected)

(10 rows affected)

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.047

Id,Dat,Cat,Val
1,2022-01-01,A,22
2,2000-05-04,A,5
3,2015-02-05,A,0
4,2010-12-03,B,14
5,2005-08-24,B,79
6,2022-09-30,B,100
7,2022-03-15,B,43
8,2021-07-10,B,43
9,2000-05-04,C,1
10,2022-12-31,C,112


Id,Dat,Cat,Val,RowNum,Rank,DenseRank,NTile
3,2015-02-05,A,0,1,1,1,1
2,2000-05-04,A,5,2,2,2,1
1,2022-01-01,A,22,3,3,3,2
4,2010-12-03,B,14,1,1,1,1
7,2022-03-15,B,43,2,2,2,1
8,2021-07-10,B,43,3,2,2,1
5,2005-08-24,B,79,4,4,3,2
6,2022-09-30,B,100,5,5,4,2
9,2000-05-04,C,1,1,1,1,1
10,2022-12-31,C,112,2,2,2,2


Id,Dat,Cat,Val,PrevVal,NextVal,FirstVal,LastVal
3,2015-02-05,A,0,NULL,5,0,22
2,2000-05-04,A,5,0,22,0,22
1,2022-01-01,A,22,5,NULL,0,22
4,2010-12-03,B,14,NULL,43,14,100
7,2022-03-15,B,43,14,43,14,100
8,2021-07-10,B,43,43,79,14,100
5,2005-08-24,B,79,43,100,14,100
6,2022-09-30,B,100,79,NULL,14,100
9,2000-05-04,C,1,NULL,112,1,112
10,2022-12-31,C,112,1,NULL,1,112


Id,Dat,Cat,Val,Cnt,MinVal,MaxVal,SumVal
3,2015-02-05,A,0,3,0,22,27
2,2000-05-04,A,5,3,0,22,27
1,2022-01-01,A,22,3,0,22,27
4,2010-12-03,B,14,5,14,100,279
7,2022-03-15,B,43,5,14,100,279
8,2021-07-10,B,43,5,14,100,279
5,2005-08-24,B,79,5,14,100,279
6,2022-09-30,B,100,5,14,100,279
9,2000-05-04,C,1,2,1,112,113
10,2022-12-31,C,112,2,1,112,113


### *Tid til opgaver...*

Lav opgave 4, 5 og 6 i [opgavehæftet](Window-functions.sql).


## Hovedpointer

- Window-funktioner er et uundværligt værktøj til analytikeren
- Window-funktioner laver, for hver række, en beregning over en mængde af rækker relateret til den aktuelle række
- En window-funktion består af følgende elementer:
  - En funktion, fx `COUNT()`, `RANK()` eller `FIRST_VALUE()`
  - En specifikation af et vindue i `OVER`-delsætningen via *Partitioning*, *Odering* og *Framing*
- Window-funktioner evalueres logisk i `SELECT`- og `ORDER BY`-delsætningerne
- Der findes forskellige typer af window-funktioner, herunder funktioner til ranking, offset, aggregeringer og statistik


## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
